# Guide to Pandas

In [ ]:
# load libraries and datasets

from IPython.display import Image
import numpy as np
import pandas as pd
from pathlib import Path
import pytz
import seaborn as sns
np.random.seed(42)
path_data = Path.cwd() / 'data'
df1 = pd.read_csv(path_data / "concat_1.csv")
df2 = pd.read_csv(path_data / 'concat_2.csv')
df3 = pd.read_csv(path_data / 'concat_3.csv')
ebola = pd.read_csv(path_data / 'country_timeseries.csv', parse_dates=['Date'])
gapminder = pd.read_csv(path_data / 'gapminder.tsv', sep='\t')
person = pd.read_csv(path_data / 'survey_person.csv')
pew = pd.read_csv(path_data / 'pew.csv')
scientists = pd.read_csv(path_data / 'scientists.csv')
site = pd.read_csv(path_data / 'survey_site.csv')
survey = pd.read_csv(path_data / 'survey_survey.csv')
tesla = pd.read_csv(path_data / 'tesla_stock_yahoo.csv', parse_dates=['Date'], date_format='%Y-%m-%d')
tips = sns.load_dataset('tips')
titanic = sns.load_dataset('titanic')
visited = pd.read_csv(path_data / 'survey_visited.csv')
weather = pd.read_csv(path_data / 'weather.csv')

Load libraries

In [572]:
# Load libraries
from datetime import datetime
import numpy as np
import pandas as pd
from pathlib import Path
import seaborn as sns

# Load data
iris = sns.load_dataset("iris")
tips = sns.load_dataset('tips')
# Drop columns class and deck to avoid dealing with categories
titanic = sns.load_dataset("titanic").drop(columns=["class", "deck"])
survey_path = Path.cwd() / "data" / "survey_visited.csv"
survey = pd.read_csv(survey_path, parse_dates=["dated"], date_format="%Y-%m-%d")

## Contents
- [Attributes and Methods](#attributes_and_methods)
- [Data Types](#data_types)
- [`Series` Objects](#series_objects)
- [`DataFrame` Objects](#dataframe_objects)
- [Select Column, Rows, and Cells](#select_columns_rows_and_cells)
- [Missing Values: `NaN`](#missing_values_nan)
- [Grouping](#grouped_and_aggregated_calculations)
- [Concatenation: Adding Row(s) or Columns(s) to Dataframe](#concatenation_adding_rows_or_columns_to_dataframe)
- [Merging Datasets](#merging_datasets)
- [Tidy Data](#tidy_data)
- [Dealing with `datetime` Objects](#dealing_with_datetime_objects)
- [Broadcasting: Dealing with Different Sized Objects](#Broadcasting:_Dealing_with_Different_Sized_Objects)
- [Apply Function Across Each Row or Column](#apply_function_across_each_row_or_column)
- [References](#references)



Difference between str.replace and .replace : 
https://stackoverflow.com/questions/56625031/what-is-the-difference-between-series-replace-and-series-str-replace

In [ ]:
## WHAT IS THIS FOR????

df['column name'] = df["column name"].replace({"part of string":"new string"}, regex=True)
# replacing values in df
# replace values in whole df : replaces exact value matches with ‘a’ to ‘b’ and ‘y’ with ‘z’
# add arg regex=True to match partial string
df.replace({'a': 'b', 'y': 'z'})
# replace values in multiple columns with same value : looks for the value 1 in column ‘a’ and ‘z’ in column ‘b’ and replaces these values with whatever is specified in value
df.replace({'a': 1, 'b': 'z'}, value='replace')
# replace values in multiple columns with multiple values : looks in column 'a' for pattern and replaces it with replace and column 'b' and replaces 'c' with 'z'
replacement = {'pattern':'replace'}
df.replace({'a':replacement, 'b':{'c':'z'}}, regex=True, inplace=True)

<a id='attributes_and_methods'></a>
## Attributes and Methods

### Attributes
- **DO NOT HAVE parentheses** >> think of as properties of object

Examples
- `.shape` returns the number of rows and columns as tuple (rows, columns) and will never change unless the data is changed
- `.columns` returns the column names

### Methods
- **HAVE parentheses** >> think of as performing some calculation or operation on object, so args can be passed into parenthesis

Examples
- `.head()` returns first n rows of DataFrame, and if no arg is passed, the default is 5
- `.info()` returns info about dataframe, args can alter how the output information is processed
    
*WARNING `.loc[]` and `.iloc[]` look similar but use subsetting synthax, like getting first element in Python list: `things[0]`*

<a id='data_types'></a>
## Data Types
|Pandas types | Python type | Description |
| ---         | ---         | --- |
| object      | string      | Most common data type |
| int64       | int         | Whole numbers |
| float64     | float       | Numbers with decimals |
| datetime64  | datetime    | datetime is found in the Python standard library <br /> (not loaded by default and needs to be imported) |

<a id='series_objects'></a>
## `Series` Objects

### What is a `Series` Object?

**`Series`** object is a one-dimensional container (like a Python built-in list)
    - First column contains indices (get by `.index`, default incrementally increases by 1 starting from 0)
    - Second column contains values (get by `.values`)
- A series is like a list with every element labelled
- To convert a `Series` object into a `DataFrame` object, wrap column part in square brackets (pandas interprets the list as a request for multiple columns, even if it only contains one)

Common way to create a series is by passing a list of elements:

In [5]:
# Create series by passing list of elements and indices (optional)
pd.Series(["Math", "Science"], index=["Class1", "Class2"])

Class1       Math
Class2    Science
dtype: object

- Every item in a list *should* be the same variable value (in the case they are added to a DataFrame)

### How are `Series` Operated on: Vectorization

Many operations are vectorized, meaning it runs like a `for` loop on each element

Going to setup two series from the `iris` dataframe to use for examples (`.loc` explained later)

In [323]:
# Both series contain floats
sepal_lengths = iris.loc[:, "sepal_length"]
petal_lengths = iris.loc[:, "petal_length"]

Scalar vectorization: for each element in the series, a scalar is applied (example below adds 3.5)

In [310]:
# 3.5 is added to every element and the new value is returned
(sepal_lengths + 3.5).head(3)

0    8.6
1    8.4
2    8.2
Name: sepal_length, dtype: float64

Series + series vectorization: for each element in both series, an operation is performed (example below adds up values)

In [ ]:
# Every element in the same position are added together, the result is returned
(sepal_lengths + petal_lengths).head(3)

0    6.5
1    6.3
2    6.0
dtype: float64

- Both series **MUST** be the same length

Equality checking vectorization: for each element in the series, a boolean is returned based on the equality check (the `.mean()` method also adds up all values in the series and divides by n to return a single value)

In [ ]:
# Every element checked whether it's greater than the mean of all values
# A boolean is returned
(sepal_lengths > sepal_lengths.mean()).head(3)

0    False
1    False
2    False
Name: sepal_length, dtype: bool

To keep only the values that were `True` from the example above, the boolean needs to be passed into a `.loc[]`

In [317]:
# Every element checked whether it's greater than the mean of all values
# A boolean is returned and only rows where True was present are retained
sepal_lengths.loc[sepal_lengths > sepal_lengths.mean()].head(3)

50    7.0
51    6.4
52    6.9
Name: sepal_length, dtype: float64

- Notice the index number starts at 50 instead of 0 now, meaning all values from index 0 to 49 were `False`
- Since `iris` is actually a dataframe, the following code does the same exact thing `iris.loc[iris["sepal_length"] > iris["sepal_length"].mean(), "sepal_length"]`
    - `.query` does the same thing as the code above and while it may be easier to read, it is not as robust so it is not recommended > `iris.query("sepal_length > sepal_length.mean()")["sepal_length"]`

In fact, any array of boolean values can be put on a series to retain values where `True` is present

In [ ]:
# For every element in a boolean array, only True values are retained
manual_bool_values = [True, True, False, False, False, False, False, True]
sepal_lengths[:8].loc[manual_bool_values]

0    5.1
1    4.9
7    5.0
Name: sepal_length, dtype: float64

- Since the boolean array has `True` values at index 0, 1, and 7, only elements at index 0, 1, and 7 from the series will be retained

<a id='dataframe_objects'></a>
## `DataFrame` Objects

### What is a `DataFrame` Object?
- Each column in DataFrame is a Series (think of as dictionary of Series objects, with column names as the keys and the Series as the values)
- After loading a DataFrame, it is recommended to understand it and fix errors before doing anything with it

### Creating `DataFrame` Objects

Create a dataframe by passing a dictionary

In [458]:
# New dataframe is created by passing a dictionary
pd.DataFrame({
    "Occupation": ["Chemist", "Statistician"],
    "Born": ["1920-07-25", "1876-06-13"],
    "Died": ["1958-04-16", "1937-10-16"],
    "Age": [37, 61]},
    index=["Rosaline Franklin", "William Gosset"])

,Occupation,Born,Died,Age
Rosaline Franklin,Chemist,1920-07-25,1958-04-16,37
William Gosset,Statistician,1876-06-13,1937-10-16,61


- This is a common way to create a dataframe
- Each key is the name of a column, which is a type of variable
- Each value for a given key is a list of values, which are values of those variables
- Can optionally set the index, which are row labels (default is incremental integers from 0)

Create an empty dataframe with `pd.DataFrame()`

In [20]:
# Create empty DataFrame
print(pd.DataFrame())

Empty DataFrame
Columns: []
Index: []


- Columns can be added optionally by using `columns` arg
- Indices can be added optionally by using `index` arg

<a id='select_columns_rows_and_cells'></a>
## Select Columns, Rows, and Cells

- `.loc[]` selects data based on column or row names
    - The order is rows then columns, `.loc[rows, columns]`
- Slicing can be used (both start and stop elements **are** included)
* `.iloc[]` is depreciated as of version 2.2.0, so do not use anymore

Explicit is better than implicit, which is why it is recommended to use `.loc`

### Selecting Columns

Select single column by column name

In [324]:
# Select all rows (:) and given column (by name)
iris.loc[:, "sepal_length"].head(3)

0    5.1
1    4.9
2    4.7
Name: sepal_length, dtype: float64

- Returns a `Series`

*Note: `iris["sepal_length"]` IS equivalent to the code above, but it is not explicit whether `sepal_length` is a row or column name*

Select multiple columns by column names

In [325]:
# Select all rows (:) and given columns by column names
columns = ["sepal_length", "petal_length"]
iris.loc[:, columns].head(3)

,sepal_length,petal_length
0,5.1,1.4
1,4.9,1.4
2,4.7,1.3


- Returns a `DataFrame`

*Note: `iris[["sepal_length", "petal_length"]]` IS equivalent to the code below, but it is not explicit whether these are row or column names*

Select multiple columns by slicing

In [327]:
# Select all rows (:) and columns from the first to (and include) the second
iris.loc[:, "sepal_length":"petal_length"].head(3)

,sepal_length,sepal_width,petal_length
0,5.1,3.5,1.4
1,4.9,3.0,1.4
2,4.7,3.2,1.3


- When slicing, column names do not need to be placed in brackets (just like the rows slicing does not)

*Note: contrary to usual python slices, **both** the start and the stop are included*

### Selecting Rows

Select single row by row index name

In [63]:
# Select all values in given row by row name
iris.loc[0, :]

sepal_length       5.1
sepal_width        3.5
petal_length       1.4
petal_width        0.2
species         setosa
Name: 0, dtype: object

- Returns a `Series` since only 1 row is returned
- To return as a `DataFrame`, wrap row part in square brackets (pandas interprets the list as a request for multiple rows, even if it only contains one)

Select multiple rows by row index names

In [328]:
# Select all values in given rows by row names
iris.loc[[0, 5], :]

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
5,5.4,3.9,1.7,0.4,setosa


- Returns a `DataFrame` since multiple rows are returned

Select rows based on Boolean conditions

In [346]:
(iris.loc[iris.loc[:, "sepal_length"] > iris.loc[:, "sepal_length"].mean(), :]
 .head(3))

,sepal_length,sepal_width,petal_length,petal_width,species
50,7.0,3.2,4.7,1.4,versicolor
51,6.4,3.2,4.5,1.5,versicolor
52,6.9,3.1,4.9,1.5,versicolor


- This code says, using the `iris` dataset, calculate the mean using all the values in the column `sepal_length`. Then, for each row of the dataframe, if the value in `sepal_length` is greater than the mean, mark it is as `True` or if it is not, mark it as `False`. Then, keep all of the rows where it is `True` and retain all columns

*Note: `iris[iris["sepal_length"]>iris["sepal_length"].mean()]` IS equivalent, but it is not explicit whether `sepal_length` is a row or column name*

### Selecting by Columns and Rows Simultaneously

Select single value by it's row and column position

In [79]:
# Select a single value by row and column name
iris.loc[0, "sepal_length"]

np.float64(5.1)

- Returns a single value
- To return as a `Series`, wrap either the column or row parts in square brackets
- To return as a `DataFrame`, wrap both column and row parts in square brackets

Select multiple rows or columns by row or column values after setting a column(s) as the index

In [331]:
iris.set_index("species").loc["setosa", :].head(3)

,sepal_length,sepal_width,petal_length,petal_width
species,,,,
setosa,5.1,3.5,1.4,0.2
setosa,4.9,3.0,1.4,0.2
setosa,4.7,3.2,1.3,0.2


- Use `.reset_index()` if index needs to be reset

### Depreciated Methods and Functions

***`.iloc[]`*** has been depreciated, but here are examples previously given

Select single column by index number

```python
# Select all rows (:) and given column (by index number)
iris.iloc[:, 0]
```
- Returns the column as a `Series`

*Note: `iris.iloc[0]` IS NOT equivalent to the code below, this is why explicit is important*

Select multiple columns by index numbers

```python
# Select all rows (:) and given columns by index numbers
iris.iloc[:, [0, 2]]
```

Select single row by row index name

```python
# Select all values in given row by index number
iris.iloc[0]
```

Select multiple rows by row index names
```python
# Select all values in given rows by index numbers
iris.iloc[[0, 5]]
```

Select single value by it's row and column position
```python
# Select a single value by row and column index numbers
iris.iloc[0, 0]
```

<a id='missing_values_nan'></a>
## Missing Values: `NaN`

Missing values are displayed as `NaN` in the data and they are `np.nan` objects

*Note: `np.NAN` and `np.NaN` were removed in the Numpy 2.0 release*

`np.nan` type is a float

In [112]:
type(np.nan)

float

### `NaN` Has no Concept of Equality

`NaN` does not equal `False`

In [ ]:
np.nan == False

False

`Nan` does not equal 0

In [332]:
np.nan == 0

False

`NaN` does not equal an empty string

In [333]:
np.nan == ''

False

`NaN` does not equal `None`

In [335]:
np.nan == None

False

`NaN` does not even equal `NaN`

In [337]:
np.nan == np.nan

False

### Dealing with `NaN` When Loading in csv Files

Convert all `NaN` values into empty strings

In [460]:
pd.read_csv(survey_path, keep_default_na=False).tail(3)

,ident,site,dated
5,752,DR-3,
6,837,MSK-4,1932-01-14
7,844,DR-1,1932-03-22


Convert all the values passed into the `na_values` arg into `NaN`

In [461]:
na_values = ["DR-1", 837]
pd.read_csv(survey_path, na_values=na_values).tail(3)

,ident,site,dated
5,752.0,DR-3,NaN
6,NaN,MSK-4,1932-01-14
7,844.0,NaN,1932-03-22


### Check if `NaN` Values Exist

Check number of `NaN` in each column using `.info()`

In [260]:
titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   who          891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
 11  alive        891 non-null    object 
 12  alone        891 non-null    bool   
dtypes: bool(2), float64(2), int64(4), object(5)
memory usage: 78.4+ KB


- The `RangeIndex` shows there are 891 rows, the `age` column shows 714 rows are non-null

Check if any values in an array are `NaN`

In [343]:
# Row 888 returns a True value, meaning there is a NaN in that row
pd.isnull(titanic.loc[:, "age"]).tail(3)

888     True
889    False
890    False
Name: age, dtype: bool

- Every value in the array is checked and if it is null, it returns `False`, otherwise it returns `True`
- Use `pd.notnull()` to conversely check if values are not null

### Fill in `NaN` Values

Replace all `NaN` values with given argument

In [345]:
# All NaN values replaced with string "XXX"
titanic.fillna("XXX").tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
888,0,3,female,XXX,1,2,23.45,S,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True


- Notice the value in column `age` in row 888 was replaced with the string `XXX`
- Passing in argument `method` is depreciated, use other `.ffill()` and `.bfill()` methods below

Fill in all `NaN` values with the value in the row above it using `.ffill()`

In [ ]:
# All NaN values replaced with the value in the row above it
titanic.ffill().tail(4)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
887,1,1,female,19.0,0,0,30.00,S,woman,False,Southampton,yes,True
888,0,3,female,19.0,1,2,23.45,S,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True


- Notice the value in column `age` in row 888 was replaced with the value in row 887
- If there are no values in all the rows above it, it remains `NaN`

Fill in all `NaN` values with the value in the row after it using `.bfill()`

In [354]:
# All NaN values replaced with the value in the row below it
titanic.bfill().tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
888,0,3,female,26.0,1,2,23.45,S,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True


- Notice the value in column `age` in row 888 was replaced with the value in row 889

Fill in `NaN` values in specified column(s) only

In [357]:
# All NaN values in column age is replaced with 999
titanic.fillna(value={"age":999}).tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
888,0,3,female,999.0,1,2,23.45,S,woman,False,Southampton,no,False
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True


- Notice the value in column `age` in row 888 was replaced with `999.0`
- Add arg `inplace=True` to modify the original dataframe

### Remove `NaN`

Drop entire row if `NaN` values in any column

In [358]:
titanic.dropna().tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
887,1,1,female,19.0,0,0,30.00,S,woman,False,Southampton,yes,True
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True


- Notice index 888 is missing since that row was dropped
- Add arg `subset=[]` with list of column names to apply drop only to given column names
- Add arg `how` to define amount of elements to drop
- Add arg `axis=0` or `axis="index"` to drop row, or `axis=1` or `axis="columns"` to drop column

### Performing Calculations When `NaN` is Present
- When a calculation is performed with `NaN` values present, it typically returns `NaN` unless built-in methods ignore missing values
    - e.g., `skipna=True` argument in `mean()` and `sum()` will not include `NaN` in their calculations

Multiplying each element in the `age` column by 100 returns `NaN` if it is `NaN` in that row

In [360]:
# `.assign` temporarily adds a column to avoid adding it permanently 
titanic.assign(age2=titanic.loc[:, "age"] * 100).tail(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone,age2
888,0,3,female,NaN,1,2,23.45,S,woman,False,Southampton,no,False,NaN
889,1,1,male,26.0,0,0,30.00,C,man,True,Cherbourg,yes,True,2600.0
890,0,3,male,32.0,0,0,7.75,Q,man,True,Queenstown,no,True,3200.0


- Notice how the value in column `age2` in row 888 is `NaN` because the value in column `age` is `NaN`

### Add `NaN` When Re-indexing Dataframe
- Common usage is when index represents some kind of interval and want to add missing values in-between

Example: find the mean fares amounts for every age in the `titanic` dataset

In [387]:
# Added `.round()` to round ages to integers (`.groupby` is explained below)
fares_by_age = titanic.round({"age":0}).groupby(["age"])["fare"].agg("mean")
fares_by_age.tail(3)

age
71.0    42.0792
74.0     7.7750
80.0    30.0000
Name: fare, dtype: float64

- Notice that there are certain ages of people that were not on the titanic

Create missing ages within an interval and assign them as `NaN`

In [366]:
# Create NaN values for missing ages between 74 and 80
fares_by_age.reindex(range(74, 81))

age
74     7.775
75       NaN
76       NaN
77       NaN
78       NaN
79       NaN
80    30.000
Name: fare, dtype: float64

- Since the data only contained ages 74 and 80, ages 75-79 are assigned `NaN`

### Depreciated Methods and Functions

***`.interpolate()`***

Fill in all `NaN` values with values in the middle of values between it
```python
titanic.interpolate().tail()
```
- If the ages on either side of a `NaN` were 18 and 20, the `NaN` would be replaced with 19

<a id='grouped_and_aggregated_calculations'></a>
## Grouping

Grouping is based on a "split-apply-combine" workflow
- Split means splitting up the data into groups
- Apply means to apply a function to each group
- Combine means to combine the results into a new dataset

### Grouping: Split

Group data by a single column

In [411]:
titanic.groupby("sex")

- Grouping returns the data type (a `DataFrameGroupBy` object) and the memory location

Goup data by multiple columns by passing a list

In [412]:
titanic.groupby(["sex", "age"])

Return all data from a single group

In [415]:
# Returns all rows where the sex is male
titanic.groupby("sex").get_group("male").head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,man,True,Southampton,no,False
4,0,3,male,35.0,0,0,8.0500,S,man,True,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,man,True,Queenstown,no,True


Iterate through every group

In [410]:
# Split dataset into groups by sex
for group in titanic.groupby("sex"):
    print(f"Group name: {group[0]}") # group name
    print(f"Group dataframe:\n{group[1].head(3)}\n") # dataframe

Group name: female
Group dataframe:
   survived  pclass     sex   age  sibsp  parch     fare embarked    who  \
1         1       1  female  38.0      1      0  71.2833        C  woman   
2         1       3  female  26.0      0      0   7.9250        S  woman   
3         1       1  female  35.0      1      0  53.1000        S  woman   

   adult_male  embark_town alive  alone  
1       False    Cherbourg   yes  False  
2       False  Southampton   yes   True  
3       False  Southampton   yes  False  

Group name: male
Group dataframe:
   survived  pclass   sex   age  sibsp  parch    fare embarked  who  \
0         0       3  male  22.0      1      0  7.2500        S  man   
4         0       3  male  35.0      0      0  8.0500        S  man   
5         0       3  male   NaN      0      0  8.4583        Q  man   

   adult_male  embark_town alive  alone  
0        True  Southampton    no  False  
4        True  Southampton    no   True  
5        True   Queenstown    no   True  



- Returns tuple
    - The first element is the name of group
    - The second element is the dataframe of the group

### Grouping: Apply and Combine


Apply a single function to every group and combine into a new dataset

In [403]:
# Calculate the mean fare for each group
titanic.groupby("sex")["fare"].agg("mean")

sex
female    44.479818
male      25.523893
Name: fare, dtype: float64

Apply a single CUSTOM function to every group and combine into a new dataset

In [437]:
# Apply custom function for calculating the mean manually
def calc_mean(values):
    sum = 0
    for value in values:
        sum += value
    return(sum / len(values))

titanic.groupby("sex", observed=True)["fare"].agg(calc_mean)

sex
female    44.479818
male      25.523893
Name: fare, dtype: float64

- `values` arg here are arrays of fare amounts separated by sex and is automatically passed into the function in `.agg()`
- Arg `observed=True` added due to FutureWarning

Apply a single CUSTOM function with additional arguments to every group and combine into a new dataset

In [ ]:
# Apply custom function with additional arguments
# Calculate the mean by sex then subtract by the global mean
def mean_dif(values, diff_value):
    sum = 0
    for value in values:
        sum += value
    return sum/len(values) - diff_value
titanic.groupby("sex")["fare"].agg(mean_dif, diff_value=titanic["fare"].mean())

sex
female    12.275610
male      -6.680315
Name: fare, dtype: float64

Apply multiple functions to every group and combine into a new dataset

In [416]:
# Calculate the mean and std fare for each group
titanic.groupby("sex")["fare"].agg(["mean", "std"]).head(3)

,mean,std
sex,,
female,44.479818,57.997698
male,25.523893,43.138263


Apply multiple functions to multiple columns by passing a `dict` and combine into a new dataset

In [417]:
# Calculate the mean and mean fare for each group
titanic.groupby("sex").agg({"fare":"mean", "sibsp":"median"}).head(3)

,fare,sibsp
sex,,
female,44.479818,0.0
male,25.523893,0.0


- Keys = column name while values = functions to apply

Apply a function to every group and return transformed dataframe with the same index as the original

In [418]:
# Calculate the z-score for fares based on sex's mean
titanic.groupby("sex")["fare"].transform(lambda x: (x-x.mean()) / x.std()).head(3)

0   -0.423612
1    0.462147
2   -0.630280
Name: fare, dtype: float64

- What this code does is:
    - Calculate the mean and std fare amounts for every group in column `sex` (i.e., `male` and `female`)
    - For the first row, subtract the mean fare amount for that group's sex from the fare amount
    - Then divide that amount by that group's sex's std
    - Return that amount into the same index as that row
    - Repeat for every row in the dataframe

In [441]:
# Calculate the mean age for 
titanic.groupby("sex")["age"].transform(lambda x: x.fillna(x.mean())).tail(3)

888    27.915709
889    26.000000
890    32.000000
Name: age, dtype: float64

- What this code does is:
    - Calculate the mean age for every group in column `sex` (i.e., `male` and `female`)
    - For the first row, if the value under row `age` is `NaN`, replace it with that group's sex mean
    - Return that amount into the same index as that row
    - Repeat for every row in the dataframe

Filter groups based on a condition

In [424]:
# Retain groups that have > 440 rows
titanic.groupby("sex").filter(lambda x: len(x) > 440).head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,man,True,Southampton,no,False
4,0,3,male,35.0,0,0,8.0500,S,man,True,Southampton,no,True
5,0,3,male,NaN,0,0,8.4583,Q,man,True,Queenstown,no,True
6,0,1,male,54.0,0,0,51.8625,S,man,True,Southampton,no,True
7,0,3,male,2.0,3,1,21.0750,S,child,False,Southampton,no,False


- Notice that female values are now missing, meaning there is less than 440 total rows with `female` in column `sex`

<a id='concatenation_adding_rows_or_columns_to_dataframe'></a>
## Concatenation: Adding Row(s) or Columns(s) to Dataframe

### Add Row(s) to Dataframe

Add a row(s) to a dataframe

In [ ]:
# Create copy of iris to avoid modifying original
iris_copy = iris.copy()

# Create a dataframe with single row
insert = pd.DataFrame({
    "sepal_length" : [6],
    "sepal_width" : [4],
    "petal_length" : [5],
    "petal_width" : [2],
    "species" : ["XXX"]},
    index=[len(iris)]
    )

# Add two dataframes together
pd.concat([iris, insert]).tail(3)

,sepal_length,sepal_width,petal_length,petal_width,species
148,6.2,3.4,5.4,2.3,virginica
149,5.9,3.0,5.1,1.8,virginica
150,6.0,4.0,5.0,2.0,XXX


- If the `index` arg was not specified, it would have been 0 even after the concat
    - Arg `ignore_index=True` resets all indices and rebuilds after all concatting is completed
- If the dataframe were longer, the same method would be used
- Multiple dataframes can be passed into `.concat()` and they will be added in order

Adding dataframe which do not share columns

In [508]:
# Neither dataframe has a shared column
pd.concat([iris, titanic]).loc[[0], :]

,sepal_length,sepal_width,petal_length,petal_width,species,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
0,5.1,3.5,1.4,0.2,setosa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,NaN,NaN,NaN,0.0,3.0,male,22.0,1.0,0.0,7.25,S,man,True,Southampton,no,False


- Even though none of the columns match, `titanic` is added to the end of `iris`
    - Arg `join=inner` only retains columns that are present in all dataframes (default is `join=outer`)
- Notice that there are two rows with index 0, and each has `NaN` values were the others have none

### Add Column(s) to Dataframe

Add a single column of values to a dataframe

In [509]:
# Add a new column that multiplies the index by 3
iris_copy['new_col'] = [x * 3 for x in list(range(0, len(iris_copy)))]
iris_copy.head(3)

,sepal_length,sepal_width,petal_length,petal_width,species,new_col
0,5.1,3.5,1.4,0.2,setosa,0
1,4.9,3.0,1.4,0.2,setosa,3
2,4.7,3.2,1.3,0.2,setosa,6


- The length of the data being added has to match the length of the dataframe or else a `ValueError` is returned

Add multiple columns to a dataframe

In [510]:
# Add multiple columns from another dataframe
pd.concat([iris, titanic], axis=1, ignore_index=False).head(3)

,sepal_length,sepal_width,petal_length,petal_width,species,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone
0,5.1,3.5,1.4,0.2,setosa,0,3,male,22.0,1,0,7.2500,S,man,True,Southampton,no,False
1,4.9,3.0,1.4,0.2,setosa,1,1,female,38.0,1,0,71.2833,C,woman,False,Cherbourg,yes,False
2,4.7,3.2,1.3,0.2,setosa,1,3,female,26.0,0,0,7.9250,S,woman,False,Southampton,yes,True


- Arg `axis=` tells `.concat()` to add by columns and not by rows
- Arg `ignore_index=True` resets all indices and rebuilds after all concatting is completed
- If there were columns from each dataframe with the same name, they would be added in different columns
- The length of columns do not matter, since empty rows will auto-fill with `NaN`

<a id='merging_datasets'></a>
## Merging Datasets
Instead of simply adding row(s) or column(s) by indices, combine based on common data values (known as "join")

* One-to-one merge: joins one column to another column (no duplicate values present): `site.merge(visited.loc[[0, 2, 6], ], left_on='name', right_on='site')`
* Many-to-one merge: one dataframe has key values that repeat, leftmost DataFrame data duplicated: `site.merge(visited, left_on='name', right_on='site')`
    * Pass columns names as list if merging on multiple columns, okay if names mismatch


Merge two datasets together (need to prepare some data before performing merge)

In [536]:
# Add a new unique id column to a copy of titanic
titanic_copy = titanic.copy()
titanic_copy['id'] = list(range(0, len(titanic_copy)))
titanic_copy.head(3)

# Create a new dataframe to merge
titanic_merge = pd.DataFrame({
    "ID" : [0, 50, 100],
    "name" : ["JB", "CB", "MB"]})
print(titanic_merge)

# Merge datasets
titanic_merge.merge(titanic_copy, left_on="ID", right_on="id")

    ID name
0    0   JB
1   50   CB
2  100   MB


,ID,name,survived,pclass,sex,age,sibsp,parch,fare,embarked,who,adult_male,embark_town,alive,alone,id
0,0,JB,0,3,male,22.0,1,0,7.2500,S,man,True,Southampton,no,False,0
1,50,CB,0,3,male,7.0,4,1,39.6875,S,child,False,Southampton,no,False,50
2,100,MB,0,3,female,28.0,0,0,7.8958,S,woman,False,Southampton,no,True,100


- Arg `how` determines the type of merge
    - `inner`: DEFAULT - data is only retained for keys found in both dataframes and merged from the right to left dataframe
    - `outer`': all data is retained, but for each key found in both dataframes, that data is merged
    - `left`: all data in the left dataframe is retained, but for each key found in both dataframes, that data is merged
    - `right`: all data in the right dataframe is retained, but for each key found in both dataframes, that data is merged
- Arg `on` specifies the columns to merge on if they are identical, otherwise use `left_on` and `right_on` for each key

<a id='tidy_data'></a>
## Tidy Data

* Transform wide to long format: `pd.melt(weather, id_vars=['id', 'year', 'month', 'element'])`
* Transform long to wide format: `weather_melt.pivot_table(index=['id', 'year', 'month', 'variable'], columns='element', values='value').reset_index()`
    
#### String accessor: access the values of the series as strings and apply methods to it
* Split string values in column: `scientists['Name'].str.split(' ')`
    * Argument `expand=True` returns results as DataFrame instead of Series list
    * Can use zip to assign as new columns `scientists['First'], scientists['Last'] = zip(*scientists['Name'].str.split(' '))`
        * `zip` takes set of iterators and returns an iterator that generates tuples of length n (n = shortest iterator length)
        
#### Transform wide to long format: melt

In [39]:
pew.head()

,religion,<$10k,$10-20k,$20-30k,$30-40k,$40-50k,$50-75k,$75-100k,$100-150k,>150k,Don't know/refused
0,Agnostic,27,34,60,81,76,137,122,109,84,96
1,Atheist,12,27,37,52,35,70,73,59,74,76
2,Buddhist,27,21,30,34,33,58,62,39,53,54
3,Catholic,418,617,732,670,638,1116,949,792,633,1489
4,Don’t know/refused,15,14,15,11,10,35,21,17,18,116




* Only 'religion' column is a variable, remaining column names are values related to income variable. Good for presenting data, bad for analyzing.

In [40]:
# melt dataframe
pd.melt(pew, id_vars=['religion'], var_name='income', value_name='count')

,religion,income,count
0,Agnostic,<$10k,27
1,Atheist,<$10k,12
2,Buddhist,<$10k,27
3,Catholic,<$10k,418
4,Don’t know/refused,<$10k,15
...,...,...,...
175,Orthodox,Don't know/refused,73
176,Other Christian,Don't know/refused,18
177,Other Faiths,Don't know/refused,71
178,Other World Religions,Don't know/refused,8




* Create a new df with columns 'religion', 'income', and 'count'
* Take the first value in the original df's 'religion' column and add to the first row in the new df's 'religion' column
* Take the second column name in the original df and add to the first row in the new df's 'income' column
* Take the first value in the second column name in the original df and add to the first row in the new df's 'count' column
* Next take the second value in the original df's 'religion' column and add to the second row in the new df's 'religion' column
* Take the second column name in the original df and add to the second row in the new df's 'income' column
* Take the second value in the second column name in the original df and add to the second row in the new df's 'count' column
* Repeat for all rows of the original df
* Then repeat process using the third column name in the original df until all columns are completed



#### Transform long to wide format: pivot_table()

In [41]:
weather_melt = pd.melt(weather, id_vars=['id', 'year', 'month', 'element'], var_name='day', value_name='temp')
weather_melt.head()

,id,year,month,element,day,temp
0,MX17004,2010,1,tmax,d1,NaN
1,MX17004,2010,1,tmin,d1,NaN
2,MX17004,2010,2,tmax,d1,NaN
3,MX17004,2010,2,tmin,d1,NaN
4,MX17004,2010,3,tmax,d1,NaN




* Change all variables in the 'element' column to column names (other languages call this casting or spreading) and store their values from the 'temp' column
* One main difference is `pivot_table` is a DataFrame method while `melt` is a Pandas function

In [42]:
weather_melt.pivot_table(index=['id', 'year', 'month', 'day'], columns='element', values='temp').reset_index().head()

element,id,year,month,day,tmax,tmin
0,MX17004,2010,1,d30,27.8,14.5
1,MX17004,2010,2,d11,29.7,13.4
2,MX17004,2010,2,d2,27.3,14.4
3,MX17004,2010,2,d23,29.9,10.7
4,MX17004,2010,2,d3,24.1,14.4




* Process is the opposite of melt



#### Shift values in DataFrame

In [43]:
weather.head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010,1,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
1,MX17004,2010,1,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
2,MX17004,2010,2,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010,2,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010,3,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [44]:
# each row is shifted down 1 row
weather.shift(1).head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,None,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MX17004,2010.0,1.0,tmax,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.8,NaN
2,MX17004,2010.0,1.0,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
3,MX17004,2010.0,2.0,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010.0,2.0,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# negative numbers shift row up
weather.shift(-1).head()

,id,year,month,element,d1,d2,d3,d4,d5,d6,...,d22,d23,d24,d25,d26,d27,d28,d29,d30,d31
0,MX17004,2010.0,1.0,tmin,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,14.5,NaN
1,MX17004,2010.0,2.0,tmax,NaN,27.3,24.1,NaN,NaN,NaN,...,NaN,29.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MX17004,2010.0,2.0,tmin,NaN,14.4,14.4,NaN,NaN,NaN,...,NaN,10.7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MX17004,2010.0,3.0,tmax,NaN,NaN,NaN,NaN,32.1,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MX17004,2010.0,3.0,tmin,NaN,NaN,NaN,NaN,14.2,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<a id='dealing_with_datetime_objects'></a>
## Dealing with `datetime` Objects

Convert a column to `datetime` type when loading it in

In [573]:
# Pass the column name and the string format of the date
pd.read_csv(survey_path, parse_dates=["dated"], date_format="%Y-%m-%d").info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ident   8 non-null      int64         
 1   site    8 non-null      object        
 2   dated   7 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 324.0+ bytes


- It is better to manually set the date synthax than letting it try to guess: [guide to formatted strings: `strptime`](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)

Convert a column in a dataframe to `datetime` type 

In [ ]:
df = pd.read_csv(survey_path)
df["date"] = pd.to_datetime(df["dated"], format="%Y-%m-%d")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   ident   8 non-null      int64         
 1   site    8 non-null      object        
 2   dated   7 non-null      object        
 3   date    7 non-null      datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(2)
memory usage: 388.0+ bytes


- Notice how column `dated` is an object while column `date` is datetime64

Return the current date and time as year, month, day, 24-hour, minute, second, and microsecond

In [ ]:
datetime.now()

datetime.datetime(2025, 8, 20, 16, 35, 16, 710183)

### Selecting Data by Dates

Select data for certain dates by using `.dt` accessor

In [ ]:
# Return dates with the 8th day in the year 1927
survey.loc[((survey["dated"].dt.year==1927) & (survey["dated"].dt.day==8)), :]

,ident,site,dated
0,619,DR-1,1927-02-08


- `.dt` allows python to search for datetime properties in the series values
- Since it is searching for two conditions, both need to be in a joined parenthesis, along with their own separate parenthesis

Select data for certain dates by setting the date column as the index



#### Subset by specific times

Calculate the difference in number of days between two dates

In [579]:
# Calculate the number of days after the oldest date
survey["date_diff"] = survey["dated"] - survey["dated"].min()
survey_copy = survey.set_index(survey["date_diff"]).copy()
survey_copy.head(3)

,ident,site,dated,date_diff
date_diff,,,,
0 days,619,DR-1,1927-02-08,0 days
2 days,622,DR-1,1927-02-10,2 days
4351 days,734,DR-3,1939-01-07,4351 days


In [598]:
survey_copy.loc[survey_copy.index > pd.Timedelta('3 days'), :]

,ident,site,dated,date_diff
date_diff,,,,
4351 days,734,DR-3,1939-01-07,4351 days
1069 days,735,DR-3,1930-01-12,1069 days
1114 days,751,DR-3,1930-02-26,1114 days
1801 days,837,MSK-4,1932-01-14,1801 days
1869 days,844,DR-1,1932-03-22,1869 days


In [596]:
print(survey_copy.index)

TimedeltaIndex([   '0 days',    '2 days', '4351 days', '1069 days',
                '1114 days',         NaT, '1801 days', '1869 days'],
               dtype='timedelta64[ns]', name='date_diff', freq=None)


In [49]:
# use query
tesla.query('ref_date.dt.days<5')

,Date,Open,High,Low,Close,Adj Close,Volume,ref_date
ref_date,,,,,,,,
0 days,2010-06-29,19.000000,25.00,17.540001,23.889999,23.889999,18766300,0 days
1 days,2010-06-30,25.790001,30.42,23.299999,23.830000,23.830000,17187100,1 days
2 days,2010-07-01,25.000000,25.92,20.270000,21.959999,21.959999,8218800,2 days
3 days,2010-07-02,23.000000,23.10,18.709999,19.200001,19.200001,5139800,3 days


In [ ]:
#### Extracting parts of date
* Extract year from single date: `pd.to_datetime('2016-02-29').year` (or directly from column with dt accessor: `ebola['Date'].dt.year`)
* Extract month from single date: `pd.to_datetime('2016-02-29').month`(or directly from column with dt accessor: `ebola['Date'].dt.month`)
* Extract day from single date: `pd.to_datetime('2016-02-29').day`(or directly from column with dt accessor: `ebola['Date'].dt.day`)

#### Create dates
* Create range of dates: `pd.date_range(start='2000-01-01', end='2000-01-05')`
    * Can set freq (default='D' for days) https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects

#### Calculations on dates
* Calculate days passed between dates: `ebola['Date'] - ebola['Date'].min()`
    * Returns `timedelta` objects, think of as time lengths
* Calculate based on different time frequencies, if index is datetime: `ebola.resample('M').mean()`
* Calculate based on different time frequencies, if datetime is in column: `ebola.reset_index().groupby(pd.Grouper(key='date', freq='1M')).mean()`

#### Dealing with time zones
* Use library: `import pytz`
* Create time with timezone: `pd.Timestamp('2017-08-29 07:00', tz='US/Eastern')`
* Add timezone to time: `pd.Timestamp('2017-08-29 09:57').tz_localize('US/Pacific')`
* Convert timezones: `pd.Timestamp('2017-08-29 09:57', tz='US/Pacific').tz_convert('US/Eastern')`
* Find durations (timezones MUST match): `pd.Timestamp('2017-08-29 12:57', tz='US/Eastern') - pd.Timestamp('2017-08-29 07:00', tz='US/Eastern')`



<a id='Broadcasting:_Dealing_with_Different_Sized_Objects'></a>
## Broadcasting: Dealing with Different Sized Objects

In [50]:
# Series and Series: match by index, remaining indexes filled with NaN
scientists['Age'] + pd.Series([1, 100])

0     38.0
1    161.0
2      NaN
3      NaN
4      NaN
5      NaN
6      NaN
7      NaN
dtype: float64

In [51]:
# Series and other objects(np.array): ValueError
# scientists['Age'] + np.array([1, 100])

In [52]:
# DataFrame and boolean vector: IndexError



<a id='apply_function_across_each_row_or_column'></a>
## Apply Function Across Each Row or Column

* "Applies" function to each element individually either by rows or columns
* Similar to writing a for loop across each row or column
* Parentheses () for the passed function NOT needed (current element automatically assigned as 1st argument in passed function)
* If passed function takes multiple arguments, pass as keywords in `.apply()`



#### `.apply()` on Series objects

* Each element assigned as first argument in passed function

In [53]:
# glance at df
tips.head()

,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


In [54]:
# custom function to calculate squares
def square(x):
return x ** 2
tips['total_bill'].apply(square).head()

0    288.6601
1    106.9156
2    441.4201
3    560.7424
4    604.6681
Name: total_bill, dtype: float64



* For each element in the series (value in DataFrame column 'total_bill'), it is passed into the `square` function and the square of that element is calculated
* Read as "apply given function to 1st element in series, then 2nd element, and so on until the last element in series"

In [55]:
# for simple functions, use lambda (previous square function in lambda form)
tips['total_bill'].apply(lambda x: x**2).head()

0    288.6601
1    106.9156
2    441.4201
3    560.7424
4    604.6681
Name: total_bill, dtype: float64

In [56]:
# okay to pass multiple args in lambda
tips['total_bill'].apply(lambda x, e=3: x**e).head()

0     4904.335099
1     1105.507304
2     9274.236301
3    13278.380032
4    14868.788579
Name: total_bill, dtype: float64



#### `.apply()` on DataFrame objects

* Column-wise: each column assigned as first argument of passed fuction
    * `axis=0` (default argument)
* Row-wise: each row assigned as first argument of passed function
    * `axis=1`

In [57]:
# column-wise apply
tips.iloc[:5, :2].apply(sum) # axis=0 is default arg

total_bill    96.61
tip           13.09
dtype: float64



* First column in DataFrame is passed as argument in apply function, and so on until all columns applied to

In [58]:
# row-wise apply
tips.iloc[:5, :2].apply(sum, axis=1)

0    18.00
1    12.00
2    24.51
3    26.99
4    28.20
dtype: float64



* First row in DataFrame is passed as argument in apply function, and so on until all rows applied to

<a id='references'></a>
## References
- [Pandas API reference](https://pandas.pydata.org/pandas-docs/stable/reference/index.html)
- "Pandas for Everyone" by Daniel Y. Chen (2018)